In [161]:
import sys
sys.path.append('..')
from spyral.core.constants import QBRHO_2_P
from spyral.core.cluster import Cluster
from spyral.solvers.solver_interp_leastsq import Guess, interpolate_trajectory
from spyral.interpolate.track_interpolator import create_interpolator
from e20009_phases.InterpLeastSqSolverPhase import fit_model_interp as interp_with_least_sq
from spyral.core.run_stacks import form_run_string

from e20009_phases.config import  DetectorParameters
from spyral_utils.nuclear import NuclearDataMap
# from spyral.solvers.solver_interp_leastsq import solve_physics_interp as local_solver


import polars as pl
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib widget

from e20009_phases.InterpLeastSqSolverPhase import solve_physics_interp as solve_with_least_sq
from e20009_phases.InterpSolverPhase import solve_physics_interp as solve_with_dumb_fit
from e20009_phases.InterpSolverPhase import fit_model_interp as interp_with_dumb_fit


In [162]:
det_params = DetectorParameters(
    magnetic_field=3.0,
    electric_field=60000.0,
    detector_length=1000.0,
    beam_region_radius=20.0,
    drift_velocity_path=Path(
        "C:\\Users\\schaeffe\\Desktop\\e20009-analysis\\e20009_parameters\\drift_velocity.csv"
    ),
    get_frequency=3.125,
    garfield_file_path=Path(
        "/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/e20009_efield_correction.txt"
    ),
    do_garfield_correction=False,
)

In [163]:


run_number = 297
event_number = 590868

In [164]:
## Proton Cluster for event  ##

cluster_index = 0
cluster_data = np.loadtxt("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10_d_p\\14_5_16_MeV_4_track\\297_590868_hand_clustered\\cluster0_proton.txt", delimiter=",")
cluster = Cluster(event_number, cluster_index, cluster_data)

# ## estimation_with_fake_input_wSQRTDEDX.ipynb ##

# proton_polar_guess = 66.74317123 * np.pi/180.0   # Radians
# proton_azimuthal_guess = 74.588245883  * np.pi/180.0   # Radians
# proton_brho_guess = 0.4392254615  # Tm
# proton_xvert_guess = 1.770714813  # mm
# proton_yvert_guess = -0.484343803  # mm
# proton_zvert_guess = 278.2503967   # mm
# guess = Guess(proton_polar_guess, proton_azimuthal_guess, proton_brho_guess, proton_xvert_guess, proton_yvert_guess, proton_zvert_guess)
# print(guess)

workspace_path= Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10_d_p\\14_5_16_MeV_4_track\\297_590868_hand_clustered")
estimate_file_path = workspace_path / f"{form_run_string(run_number)}.parquet"
estimate_df = pl.scan_parquet(estimate_file_path)
proton_cluster_index = 0
estimate_df = estimate_df.filter((pl.col("event") == event_number) & (pl.col("cluster_index") == proton_cluster_index)).collect()
# estimate_df = estimate_df.filter(pl.col("event") == event_number).collect()
print(estimate_df)
guess = Guess(estimate_df['polar'][0], estimate_df['azimuthal'][0], estimate_df['brho'][0], estimate_df['vertex_x'][0], estimate_df['vertex_y'][0], estimate_df['vertex_z'][0])
print(guess)


shape: (1, 23)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ event  ┆ cluster_in ┆ cluster_la ┆ ic_amplit ┆ … ┆ sqrt_dEdx ┆ dE        ┆ arclength ┆ direction │
│ ---    ┆ dex        ┆ bel        ┆ ude       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ i64    ┆ ---        ┆ ---        ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ i64       │
│        ┆ i64        ┆ i64        ┆ f64       ┆   ┆           ┆           ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 590868 ┆ 0          ┆ 0          ┆ 945.0     ┆ … ┆ 10.328648 ┆ 2702.1796 ┆ 25.32954  ┆ 0         │
│        ┆            ┆            ┆           ┆   ┆           ┆ 76        ┆           ┆           │
└────────┴────────────┴────────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘
Guess(polar=0.646019784421105, azimuthal=4.636141764209904, brho=0.756047274

In [165]:
## Triton Cluster ## 

# cluster_index = 3
# cluster_data = np.loadtxt("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10(d,p)\\14_5_16_MeV_4_track\\256_597527\\cluster3.txt", delimiter=",")
# cluster = Cluster(event_number, cluster_index, cluster_data)

# ## estimation_with_fake_input_wSQRTDEDX.ipynb ##

# triton_polar_guess = 15.73605101 * np.pi/180.0   # Radians
# triton_azimuthal_guess = 320.899109 * np.pi/180.0   # Radians
# triton_brho_guess = 1.187993438   # Tm
# triton_xvert_guess = -1.532697897  # mm
# triton_yvert_guess = -1.884500457  # mm
# triton_zvert_guess = 271.9177137  # mm

# guess = Guess(triton_polar_guess, triton_azimuthal_guess, triton_brho_guess, triton_xvert_guess, triton_yvert_guess, triton_zvert_guess)
# print(guess)

In [166]:
# Get drift velocity

dv_lf: pl.LazyFrame = pl.scan_csv(det_params.drift_velocity_path)
dv_df: pl.DataFrame = dv_lf.filter(pl.col("run") == run_number).collect()
mm_tb: float = dv_df.get_column("average_micromegas_tb")[0]
w_tb: float = dv_df.get_column("average_window_tb")[0]
mm_err: float = dv_df.get_column("average_micromegas_tb_error")[0]
w_err: float = dv_df.get_column("average_window_tb_error")[0]

# print(dv_df)

In [167]:
z = 1 # atomic number

a_p = 1
a_t = 3 # mass number
nuclear_map = NuclearDataMap()
proton = nuclear_map.get_data(z, a_p)
# triton = nuclear_map.get_data(z, a_t)
# print(nucleus.A)

In [168]:
        # Result storage
phys_results: dict[str, list] = {
            "event": [],
            "cluster_index": [],
            "cluster_label": [],
            "vertex_x": [],
            "sigma_vx": [],
            "vertex_y": [],
            "sigma_vy": [],
            "vertex_z": [],
            "sigma_vz": [],
            "brho": [],
            "sigma_brho": [],
            "ke": [],
            "sigma_ke": [],
            "polar": [],
            "sigma_polar": [],
            "azimuthal": [],
            "sigma_azimuthal": [],
            "redchisq": [],
        }

In [169]:
path_to_ODE_solution_mesh = Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\InterpSolver_assets\\1H_in_2H2_600Torr.npy")    # Mesh for protons 
# path_to_ODE_solution_mesh = Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\InterpSolver_assets\\3H_in_2H2_600Torr.npy")    # Mesh for tritons
tracks = create_interpolator(path_to_ODE_solution_mesh)

In [170]:

print(tracks.polar_bins)

360


In [171]:
solve_with_dumb_fit(
    run_number,
    event_number,
    proton_cluster_index,
    cluster,
    guess,
    proton,
    tracks,
    det_params,
    w_tb,
    mm_tb,
    w_err,
    mm_err,
    phys_results,
)

# Write out the results
physics_df = pl.DataFrame(phys_results)
output_file_path = workspace_path / f"run_{run_number:04d}_{proton.isotopic_symbol}.parquet"
physics_df.write_parquet(output_file_path)

In [172]:
result = interp_with_dumb_fit(cluster, guess, proton, tracks, det_params, w_tb, mm_tb, w_err, mm_err)
if result is None:
    print('Guess outside of interpolation range!')
best_fit_trajectory = interpolate_trajectory(result, tracks, proton)
cluster.data[:, :3] *= 0.001



[[Fit Statistics]]
    # fitting method   = L-BFGS-B
    # function evals   = 770
    # data points      = 1
    # variables        = 6
    chi-square         = 6.6236e-06
    reduced chi-square = 6.6236e-06
    Akaike info crit   = 0.07512290
    Bayesian info crit = -11.9248771
##  Warning: uncertainties could not be estimated:
[[Variables]]
    brho:        0.62950369 (init = 0.7560473)
    polar:       0.76729566 (init = 0.6460198)
    vertex_rho:  0.01477219 (init = 0.01379795)
    vertex_phi:  4.18709320 (init = 3.065307)
    vertex_x:   -0.00740780 == 'vertex_rho * cos(vertex_phi)'
    vertex_y:   -0.01278054 == 'vertex_rho * sin(vertex_phi)'
    vertex_z:    0.41347097 (init = 0.3568465)
    azimuthal:   4.68591179 (init = 4.636142)


In [173]:
proton_polar_final = result["polar"].value * 180 / np.pi
proton_azimuthal_final = result["azimuthal"].value
proton_brho_final = result["brho"].value
proton_xvert_final = result["vertex_x"].value
proton_yvert_final = result["vertex_y"].value
proton_zvert_final = result["vertex_z"].value

momentum = proton_brho_final  * QBRHO_2_P
kinetic_energy = np.sqrt(momentum**2.0 + proton.mass**2.0) - proton.mass
print(f"kinetic_energy: {kinetic_energy}")
print(f"proton_polar_final: {proton_polar_final}")

kinetic_energy: 18.791088310717555
proton_polar_final: 43.96280306912637


In [174]:
# triton_polar_final = result["polar"].value * 180 / np.pi
# triton_azimuthal_final = result["azimuthal"].value
# triton_brho_final = result["brho"].value
# triton_xvert_final = result["vertex_x"].value
# triton_yvert_final = result["vertex_y"].value
# triton_zvert_final = result["vertex_z"].value

# momentum = triton_brho_final  * QBRHO_2_P
# kinetic_energy = np.sqrt(momentum**2.0 + triton.mass**2.0) - triton.mass
# print(kinetic_energy)
# print(triton_polar_final)

In [175]:
# Plot fit
fig = make_subplots(2, 2, subplot_titles=["XY Projection", "XZ Projection", "YZ Projection"], specs=[[{"rowspan": 2}, {}],[None, {}]])
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 0],
        y=cluster.data[:, 1],
        mode="markers", 
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 0],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_x"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_x"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False,
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False
    ),
    row=2,
    col=2
)

fig["layout"]["xaxis1"]["title"] = "X (m)"
fig["layout"]["xaxis1"]["range"] = [-0.3, 0.3]
fig["layout"]["yaxis1"]["title"] = "Y (m)"
fig["layout"]["yaxis1"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis2"]["title"] = "Z (m)"
fig["layout"]["xaxis2"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis2"]["title"] = "X (m)"
fig["layout"]["yaxis2"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis3"]["title"] = "Z (m)"
fig["layout"]["xaxis3"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis3"]["title"] = "Y (m)"
fig["layout"]["yaxis3"]["range"] = [-0.3, 0.3]

fig.update_layout(
    width=1450,
    height=725
)